In [1]:
%cd ../../
%load_ext autoreload
%autoreload 2

/media/olle/DATA/Projects/orderbookrl


In [2]:
from orderbookrl.evaluate.collect_data import load_env_agent, run_through_all_data
import pandas as pd
%matplotlib inline

In [ ]:
agent_id = 'PPO'
path = 'logs/marketorderenvfunds/ppo/PPOADV_MarketOrderEnv-v0_0_2018-08-27_12-39-581hfllrq_/'
checkpoint = 40
data_dir = '/media/olle/DATA/Projects/orderbookrl/data/test/'
env, agent = load_env_agent(agent_id, path, checkpoint, data_path=data_dir)
result, trades, states, actions, rewards, quotes = run_through_all_data(env, agent)

Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:35758 to respond...


Using custom preprocessor mv


Waiting for redis server at 127.0.0.1:27896 to respond...
Starting local scheduler with the following resources: {'CPU': 8, 'GPU': 1}.

View the web UI at http://localhost:8889/notebooks/ray_ui90881.ipynb?token=5b95d0951235a905b4d1aa4b87637a2dbe75b45daa26a76e



Created LogSyncer for /home/olle/ray_results/2018-09-04_08-18-046ifbo02p -> None
Using custom preprocessor mv


/home/olle/miniconda3/envs/orderbookmdp/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning:

`nthreads` argument is deprecated, pass `use_threads` instead

/home/olle/miniconda3/envs/orderbookmdp/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning:

`nthreads` argument is deprecated, pass `use_threads` instead



2018-04-04T17:44:39.945000Z 1.0105197271229962
2018-04-04T20:53:35.204000Z 1.0417504184273332
2018-04-05T00:22:07.765000Z 1.0708257598005855


In [ ]:
actions.columns = ['BUY=0_SELL=1_HOLD=2']
result.join(actions).plot(subplots=True, figsize=(15,10));

In [ ]:
n_cols = 3
layout = (n_cols, 1+ int( (1+states.shape[1])/n_cols))
states.columns = ['ofi', 'vol_mb', 'macd', 'mid_std', 'possession']
states.plot(subplots=True, figsize=(15,10), layout=layout);

In [ ]:
mid = (quotes['ask'] + quotes['bid'])/2
mid.name = 'mid'
temp = mid.to_frame().join(result.possession)
temp = temp.join(result.capital/result.capital[0])
temp.plot(subplots=True, figsize=(10,10));

In [ ]:
port_returns = result.capital.pct_change().iloc[1:]
port_cum_ret = (1+ port_returns).cumprod()
port_cum_ret.name = 'portfolio'
cum_returns = (1 + mid.pct_change().iloc[1:]).cumprod()
cum_returns.name = 'bitcoin'
temp = pd.concat([port_cum_ret, cum_returns], axis=1)
temp.plot(figsize=(10,10));

In [ ]:
temp = mid.to_frame().join(result.possession)
temp.plot(secondary_y='possession', figsize=(15, 5));

In [ ]:
port_returns.rolling(100).mean().nlargest(n=25)

In [ ]:
temp['2018-06-02 06'].plot(subplots=True, figsize=(10,10));

In [ ]:
actions.head()

In [ ]:
states.head()

In [ ]:
states.shape

In [ ]:
from sklearn import tree
import graphviz 
import numpy as np
np.random.seed(1)
n = 10000
idx = np.random.choice(np.arange(states.shape[0]), n)
reg = tree.DecisionTreeClassifier().fit(states.iloc[idx], actions.values[idx].ravel())
dot_data = tree.export_graphviz(reg, max_depth=3, out_file=None,
                         feature_names=states.columns, class_names=['Buy', 'Sell', 'Hold'],  
                         filled=True, rounded=True,  
                         special_characters=True)  
graphviz.Source(dot_data)  